# LoRA Fine-tuning with peft

## Load libraries

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

## Load Model and Tokenizer

In [ ]:
model_name = "microsoft/deberta-v3-base"  # Example 1B param model (replace as needed)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) # num_labels should align with your dataset

## Load and Prepare Dataset


In [ ]:
dataset = load_dataset("glue", "mrpc")  # Example dataset (replace as needed)
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

## Define LoRA Configuration


In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, # Specify your task type
    inference_mode=False,
    r=8,  # Rank of LoRA matrices (common values: 8, 16, 32)
    lora_alpha=32,  # Scaling factor for LoRA updates
    lora_dropout=0.1,  # Dropout for LoRA layers
    target_modules=["query", "value"] # option to choose which modules to apply LoRA, like query, key, value, dense
)

## Peft model

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## Training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./lora_finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

## Define Trainer and train

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()
